In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-02-14 06:47:10--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-02-14 06:47:11--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucab82c64bcce3940bd708e00411.dl.dropboxusercontent.com/cd/0/inline/BI4cpeNn5B65M_thG85TRAVDZ55NpxxOYvcaz0Q_iYwzGwOcfCp4c6ZTg-ns9HEsZL6uhzYklVHMzzzngMSmASioTvOBEOGv38Kb4l-tST8IP2K3DisabZP_Pz5iL8g14rI/file# [following]
--2021-02-14 06:47:11--  https://ucab82c64bcce3940bd708e00411.dl.dropboxusercontent.com/cd/0/inline/BI4cpeNn5B65M_thG85TRAVDZ55NpxxOYvcaz0Q_iYwzGwOcfCp4c6ZTg-ns9HEsZL6uh

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"I'm not great at writing reviews, so I'll just spout my opinions...  I loved this series at first. The adventure, the action, the comedy, the drama... I thought it was all brilliant. Anderson, Tapping, Shanks, Judge, Davis... I loved them all. Davis, it seemed, was the fifth-most important person in the cast. Not a big deal. But when his character (General Hammond) left at the end of the seventh season, and Anderson's character (Colonel O'Neill) moved from the field to the office, the quality of the series suddenly fell off a cliff. I don't know whether it's because Hammond was more important that I realized or what, but for some reason, after the seventh season, the series turned to ****.  The first seven seasons, though, were awesome. The movie Stargate seemed mediocre the first time I saw it, but it turned out to be, even if this wasn't the original intention, a brilliant setup to the series. I rec

In [6]:
model = Sequential()

In [7]:
model.add(Input(shape=(1,), dtype="string"))

In [8]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [9]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens + 1, 128))

model.add(LSTM(64))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 24s 21ms/step - loss: 0.6021 - accuracy: 0.6541
Epoch 2/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4507 - accuracy: 0.7895
Epoch 3/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4173 - accuracy: 0.8120
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3957 - accuracy: 0.8249
Epoch 5/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3699 - accuracy: 0.8405
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3678 - accuracy: 0.8405
Epoch 7/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3443 - accuracy: 0.8554
Epoch 8/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3292 - accuracy: 0.8622
Epoch 9/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3164 - accuracy: 0.8678
Epoch 10/10
782/782 [==============================] - 16s 21ms/step - loss: 0.322

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 12s 14ms/step - loss: 0.5550 - accuracy: 0.7803


[0.5550280213356018, 0.7802799940109253]

In [14]:
text = "I loved the movie !"

In [15]:
model.predict([text])

array([[0.9609259]], dtype=float32)